# 从头开始​​代理执行器

在本笔记本中，我们将介绍如何构建基本代理执行器
划痕。

![图](./img/agent-executor-diagram.png)

## 设置

首先我们需要安装需要的包
```bash
yarn add @langchain/community @langchain/anthropic @langchain/langgraph @langchain/core
```

接下来，我们需要为 Anthropic（我们将使用的 LLM）和 Tavily（我们将使用的 LLM）设置 API 密钥
我们将使用的搜索工具）
```bash
export ANTHROPIC_API_KEY=
export TAVILY_API_KEY=
```

或者，我们可以设置 API 密钥
[LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。
```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=
```

## 定义图模式

我们首先需要定义图状态。传统 LangGraph 代理的状态有一个属性 `messages`，它保存原始输入以及对话历史记录。

In [21]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 定义工具

接下来，我们将定义在此 LangGraph 图中使用的工具。在此示例中，我们将使用预构建的 Tavily 搜索工具，但是您可以使用任何预构建的或您想要的自定义工具。请参阅[本指南](https://js.langchain.com/docs/how_to/custom_tools/)了解如何创建自定义 LangChain 工具。

我们希望将我们的工具传递给 `ToolNode`，这是 LangGraph 执行工具的方式。

In [ ]:
import { TavilySearch } from "@langchain/tavily";
import { ToolNode } from "@langchain/langgraph/prebuilt";

const tools = [new TavilySearch({ maxResults: 1 })];

const toolNode = new ToolNode<typeof AgentState.State>(tools);

## 定义节点

我们现在需要在图中定义一些不同的节点。在`langgraph`中，一个节点
可以是一个函数或一个
[可运行](https://js.langchain.com/docs/expression_language/)。有两个
为此我们需要的主要节点：

1. 代理：负责决定采取什么（如果有）行动。
2. 调用工具的函数：如果代理决定采取行动，该节点
然后将执行该操作。

我们还需要定义一些边。其中一些边缘可能是有条件的。
它们是有条件的原因是基于节点的输出，其中之一
可以采取多条路径。直到该节点才知道所采取的路径
运行（LLM 决定）。

1. 条件边缘：调用代理后，我们应该：如果
代理说要采取行动，那么调用工具的函数应该是
称为 b.如果代理说已经完成，那么就应该完成
2. 正常边缘：调用工具后，应始终返回到
代理决定下一步做什么

让我们定义节点以及一个函数来决定什么条件
边取。

In [23]:
import type { RunnableConfig } from "@langchain/core/runnables";
import { ChatAnthropic } from "@langchain/anthropic";
import { END } from "@langchain/langgraph";

// 定义要在代理中使用的 LLM
const llm = new ChatAnthropic({
  model: "claude-3-5-sonnet-20240620",
  temperature: 0,
}).bindTools(tools); // Ensure you bind the same tools passed to the ToolExecutor to the LLM, so these tools can be used in the agent

// 定义将用于确定要下降的条件边沿的逻辑
const shouldContinue = (data: typeof AgentState.State): "executeTools" | typeof END => {
  const { messages } = data;
  const lastMsg = messages[messages.length - 1];
  // 如果代理调用了工具，我们应该继续。如果没有，我们就可以结束了。
  if (!("tool_calls" in lastMsg) || !Array.isArray(lastMsg.tool_calls) || !lastMsg?.tool_calls?.length) {
    return END;
  }
  // 通过返回我们想要去的下一个节点的名称
  // LangGraph将自动路由到该节点
  return "executeTools";
};

const callModel = async (data: typeof AgentState.State, config?: RunnableConfig): Promise<Partial<typeof AgentState.State>> => {
  const { messages } = data;
  const result = await llm.invoke(messages, config);
  return {
    messages: [result],
  };
};


## 定义图表

我们现在可以将它们放在一起并定义图表！

In [24]:
import { START, StateGraph } from "@langchain/langgraph";

// 定义一个新图
const workflow = new StateGraph(AgentState)
  // 定义我们将在其之间循环的两个节点
  .addNode("callModel", callModel)
  .addNode("executeTools", toolNode)
  // 将入口点设置为“callModel”
  // 这意味着该节点是第一个被调用的节点
  .addEdge(START, "callModel")
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“callModel”。
    // 这意味着这些是调用“agent”节点后获取的边。
    "callModel",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinue,
  )
  // 我们现在添加从“tools”到“agent”的正常边缘。
  // 这意味着在调用“tools”之后，接下来调用“agent”节点。
  .addEdge("executeTools", "callModel");

const app = workflow.compile();

In [29]:
import { HumanMessage, BaseMessage } from "@langchain/core/messages";

let finalResult: BaseMessage | undefined;

const prettyLogOutput = (output: Record<string, any>) => {
  const keys = Object.keys(output);
  const firstItem = output[keys[0]];
  if ("messages" in firstItem) {
    console.log(`(node) ${keys[0]}:`, firstItem.messages[0]);
    console.log("----\n");
  }
}

const inputs = { messages: [new HumanMessage("Search the web for the weather in sf")] };
for await (const s of await app.stream(inputs)) {
  prettyLogOutput(s);
  if ("callModel" in s && s.callModel.messages?.length) {
    finalResult = s.callModel.messages[0];
  }
}

console.log("Final Result: ", finalResult.content)

(node) callModel:  AIMessage {
  "id": "msg_01SwpMu2zu8W4NZeLEg5DHKj",
  "content": [
    {
      "type": "text",
      "text": "Certainly! I'll use the Tavily search engine to find current weather information for San Francisco (SF). Let me search for that information for you."
    },
    {
      "type": "tool_use",
      "id": "toolu_013Asn4HooNPMtYYapQPcewB",
      "name": "tavily_search_results_json",
      "input": {
        "input": "current weather in San Francisco"
      }
    }
  ],
  "additional_kwargs": {
    "id": "msg_01SwpMu2zu8W4NZeLEg5DHKj",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 416,
      "output_tokens": 94
    }
  },
  "response_metadata": {
    "id": "msg_01SwpMu2zu8W4NZeLEg5DHKj",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 416,